In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def masked_log_softmax(vector:torch.Tensor, mask:torch.Tensor, dim:int=-1) -> torch.Tensor:
    if mask is not None:
        mask = mask.float()
        while mask.dim() < vector.dim():
            mask = mask.unsqueeze(1)
        # vector + mask.log() is an easy way to zero out masked elements in log space, but it
        # results in nans when the whole vector is masked.  We need a very small value instead of a
        # zero in the mask for these cases.  log(1 + 1e-45) is still basically 0, so we can safely
        # just add 1e-45 before calling mask.log().  We use 1e-45 because 1e-46 is so small it
        # becomes 0 - this is just the smallest value we can actually use.
        vector = vector + (mask + 1e-40).log()
    return F.log_softmax(vector, dim=dim)

def masked_max(vector:torch.Tensor, mask:torch.Tensor, dim:int, keep_dim:bool=False, min_val:float=-1e7) -> (torch.Tensor, torch.Tensor):
    """
    计算最大值：在masked值的特定的维度
    :param vector: 计算最大值的vector，假定没有mask的部分全是0
    :param mask: vector的mask，必须是可以扩展到vector的形状
    :param dim: 计算max的维度
    :param keep_dim: 是否保持dim
    :param min_val: paddings的最小值
    :return: 包括最大值的Tensor
    """
    one_minus_mask = ~mask
    replaced_vector = vector.masked_fill(one_minus_mask, min_val)
    max_value, max_index = replaced_vector.max(dim=dim, keepdim=keep_dim)
    return max_value, max_index


In [2]:
class Encoder(nn.Module):
    def __init__(self, embedding_dim, hidden_size, num_layers=1, batch_first=True, bidirectional=True):
        super(Encoder, self).__init__()
        self.batch_first = batch_first
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=batch_first, bidirectional=bidirectional)

    def forward(self, embedded_inputs, input_lengths):
        # 打包RNN的填充序列
        packed = nn.utils.rnn.pack_padded_sequence(embedded_inputs, input_lengths.cpu(), batch_first=self.batch_first)
        # forward pass through RNN
        outputs, hidden = self.rnn(packed)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=self.batch_first)
        # 返回输出和最终状态
        return outputs, hidden

In [3]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.W1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W2 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.vt = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, decoder_state, encoder_outputs, mask):
        # (batch_size, max_seq_len, hidden_size)
        encoder_transform = self.W1(encoder_outputs)

        # (batch_size, 1(unsqueezed), hidden_size)
        decoder_transform = self.W2(decoder_state).unsqueeze(1)

        # (batch_size, max_seq_len, 1) => (batch_size, max_seq_len)
        u_i = self.vt(torch.tanh(encoder_transform + decoder_transform)).squeeze(-1)

        # softmax with only valid inputs, excluding zero padded parts
        # log_softmax for a better numerical stability
        log_score = masked_log_softmax(u_i, mask, dim=-1)
        return log_score

In [4]:
class PointerNet(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_size, bidirectional=True, batch_first=True):
        super(PointerNet, self).__init__()
        # Embedding dimension
        self.embedding_dim = embedding_dim
        # decoder hidden size
        self.hidden_size = hidden_size
        # bidirectional encoder
        self.bidirectional = bidirectional
        self.num_directions = 2 if bidirectional else 1
        self.num_layers = 1
        self.batch_first = batch_first

        # 我们将嵌入层用于以后更复杂的应用程序用法，例如单词序列。
        self.embedding = nn.Linear(in_features=input_dim, out_features=embedding_dim,bias=False)
        self.encoder = Encoder(embedding_dim=embedding_dim, hidden_size=hidden_size, num_layers=self.num_layers, bidirectional=bidirectional,batch_first=batch_first)

        self.decoding_rnn = nn.LSTMCell(input_size=hidden_size, hidden_size=hidden_size)
        self.attn = Attention(hidden_size=hidden_size)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)

    def forward(self, input_seq, input_lengths):
        if self.batch_first:
            batch_size = input_seq.size(0)
            max_seq_len = input_seq.size(1)
        else:
            batch_size = input_seq.size(1)
            max_seq_len = input_seq.size(0)

        # embedding
        embedded = self.embedding(input_seq)
        # (batch_size, max_seq_len, embedding_dim)

        # encoder_output => (batch_size, max_seq_len, hidden_size) if batch_first else (max_seq_len, batch_size, hidden_size)
        # hidden_size is usually set same as embedding size
        # encoder_hidden => (num_layers * num_directions, batch_size, hidden_size) for each of h_n and c_n
        encoder_outputs, encoder_hidden = self.encoder(embedded, input_lengths)

        if self.bidirectional:
            # Optionally, Sum bidirectional RNN outputs
            # (batch_size, max_seq_len, hidden_size)
            encoder_outputs = encoder_outputs[:, :, :self.hidden_size] + encoder_outputs[:, :, self.hidden_size:]

        encoder_h_n, encoder_c_n = encoder_hidden
        # (1, 2, batch_size, hidden_size)
        encoder_h_n = encoder_h_n.view(self.num_layers, self.num_directions, batch_size, self.hidden_size)
        encoder_c_n = encoder_c_n.view(self.num_layers, self.num_directions, batch_size, self.hidden_size)

        # Let's use zeros as an initial input
        # (batch_size, hidden_size)
        decoder_input = encoder_outputs.new_zeros(torch.Size((batch_size, self.hidden_size)))
        # ((batch_size, hidden_size), (batch_size, hidden_size))
        decoder_hidden = (encoder_h_n[-1, 0, :, :].squeeze(), encoder_c_n[-1, 0, :, :].squeeze())

        # (batch_size, max_seq_len, max_seq_len)
        range_tensor = torch.arange(max_seq_len, device=input_lengths.device, dtype=input_lengths.dtype).expand(batch_size, max_seq_len, max_seq_len)
        each_len_tensor = input_lengths.view(-1, 1, 1).expand(batch_size, max_seq_len, max_seq_len)

        # (batch_size, max_seq_len, max_seq_len)
        row_mask_tensor = (range_tensor < each_len_tensor)
        col_mask_tensor = row_mask_tensor.transpose(1, 2)
        mask_tensor = row_mask_tensor * col_mask_tensor

        pointer_log_scores = []
        pointer_argmaxs = []

        for i in range(max_seq_len):
            # we will simply mask out when calculating attention or max (and loss later)
            # not all input and hidden, just for simplicity
            # (batch_size, max_seq_len)
            sub_mask = mask_tensor[:, i, :]

            # h,c is both (batch_size, hidden_size)
            h_i, c_i = self.decoding_rnn(decoder_input, decoder_hidden)

            # next hidden
            decoder_hidden = (h_i, c_i)

            # get a pointer distribution over the encoder outputs using attention
            # (batch_size, max_seq_len)
            log_pointer_score = self.attn(h_i, encoder_outputs, sub_mask)
            pointer_log_scores.append(log_pointer_score)

            # get the indices of maximum pointer
            # (batch_size, 1)
            _, masked_argmax = masked_max(log_pointer_score, sub_mask, dim=1, keep_dim=True)

            pointer_argmaxs.append(masked_argmax)

            # (batch_size, 1, hidden_size)
            index_tensor = masked_argmax.unsqueeze(-1).expand(batch_size, 1, self.hidden_size)

            # encoder_outputs为(batch_size, max_seq_len, hidden_size)
            # index为(batch_size, 1, hidden_size)，且所有hidden_size个的数据都是一样的，都是0-30的数字
            # decoder_input: (batch_size , 1, hidden_size).squeeze(1)即(batch_size, hidden_size)
            decoder_input = torch.gather(encoder_outputs, dim=1, index=index_tensor).squeeze(1)

        # stack是叠加，会增加一个维度
        # t * (batch_size, max_seq_len) t为max_seq_len, stack之后变成(batch_size, max_seq_len, max_seq_len)
        pointer_log_scores = torch.stack(pointer_log_scores, 1)
        # cat是在现有维度上续接，不会产生新维度
        # t * (batch_size, 1) cat之后变成 (batch_size, max_seq_len)
        pointer_argmaxs = torch.cat(pointer_argmaxs, 1)

        return pointer_log_scores, pointer_argmaxs, mask_tensor

In [5]:
import itertools
import numpy as np
from torch.utils.data.dataset import Dataset

from tqdm import tqdm


def tsp_opt(points):
    """
    Dynamic programing solution for TSP - O(2^n*n^2)
    https://gist.github.com/mlalevic/6222750
    :param points: List of (x, y) points
    :return: Optimal solution
    """

    def length(x_coord, y_coord):
        return np.linalg.norm(np.asarray(x_coord) - np.asarray(y_coord))

    # Calculate all lengths
    all_distances = [[length(x, y) for y in points] for x in points]
    # Initial value - just distance from 0 to every other point + keep the track of edges
    a = {(frozenset([0, idx+1]), idx+1): (dist, [0, idx+1]) for idx, dist in enumerate(all_distances[0][1:])}
    cnt = len(points)
    for m in range(2, cnt):
        b = {}
        for S in [frozenset(C) | {0} for C in itertools.combinations(range(1, cnt), m)]:
            for j in S - {0}:
                # This will use 0th index of tuple for ordering, the same as if key=itemgetter(0) used
                b[(S, j)] = min([(a[(S-{j}, k)][0] + all_distances[k][j], a[(S-{j}, k)][1] + [j])
                                 for k in S if k != 0 and k != j])
        a = b
    res = min([(a[d][0] + all_distances[0][d[1]], a[d][1]) for d in iter(a)])
    return np.asarray(res[1])


class TSPDataset(Dataset):
    """
    Random TSP dataset
    """

    def __init__(self, data_size, min_seq_len, max_seq_len, solver=tsp_opt, solve=True):
        self.data_size = data_size
        self.min_leq_len = min_seq_len
        self.max_seq_len = max_seq_len
        self.solve = solve
        self.solver = solver
        self.data = self._generate_data()

    def __len__(self):
        return self.data_size

    def __getitem__(self, idx):
        tensor = torch.from_numpy(self.data['Points_List'][idx]).float()
        length = len(self.data['Points_List'][idx])
        solution = torch.from_numpy(self.data['Solutions'][idx]).long() if self.solve else None
        return tensor, length, solution

    def _generate_data(self):
        """
        :return: Set of points_list ans their One-Hot vector solutions
        """
        points_list = []
        solutions = []
        data_iter = tqdm(range(self.data_size), unit='data')
        for i, _ in enumerate(data_iter):
            data_iter.set_description('Data points %i/%i' % (i+1, self.data_size))
            points_list.append(np.random.random((np.random.randint(self.min_leq_len, self.max_seq_len), 2)))
        solutions_iter = tqdm(points_list, unit='solve')
        if self.solve:
            for i, points in enumerate(solutions_iter):
                solutions_iter.set_description('Solved %i/%i' % (i+1, len(points_list)))
                solutions.append(self.solver(points))
        else:
            solutions = None

        return {'Points_List':points_list, 'Solutions':solutions}

    def _to1hot_vec(self, points):
        """
        :param points: List of integers representing the points indexes
        :return: Matrix of One-Hot vectors
        """
        vec = np.zeros((len(points), self.max_seq_len))
        for i, v in enumerate(vec):
            v[points[i]] = 1

        return vec


def sparse_seq_collate_fn(batch):
    batch_size = len(batch)

    sorted_seqs, sorted_lengths, sorted_label = zip(*sorted(batch, key=lambda x:x[1], reverse=True))

    padded_seqs = [seq.resize_as_(sorted_seqs[0]) for seq in sorted_seqs]

    # (sparse) batch_size X max_seq_len X input_dim
    seq_tensor = torch.stack(padded_seqs)

    # batch_size
    length_tensor = torch.LongTensor(sorted_lengths)

    padded_labels = list(zip(*(itertools.zip_longest(*sorted_label, fillvalue=-1))))

    # batch_size X max_seq_len (-1 padding)
    label_tensor = torch.LongTensor(padded_labels).view(batch_size, -1)

    # TODO: Currently, PyTorch DataLoader with num_workers >= 1 (multiprocessing) does not support Sparse Tensor
    # TODO: Meanwhile, use a dense tensor when num_workers >= 1.
    # seq_tensor = seq_tensor.to_dense()

    return seq_tensor, length_tensor, label_tensor

In [6]:
from torch.optim import Adam
import torch.backends.cudnn as cudnn
from torch.utils.data.dataloader import DataLoader

class AverageMeter:
    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.__init__()

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def masked_accuracy(output, target, mask):
    with torch.no_grad():
        masked_output = torch.masked_select(output, mask)
        masked_target = torch.masked_select(target, mask)
        accuracy = masked_output.eq(masked_target).float().mean()

        return accuracy

In [7]:
import pickle

min_length = 5
max_length = 10
batch_size1 = 256
no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
cudnn.benchmark = True if use_cuda else False

print("正在加载训练数据集")
# train_set = TSPDataset(data_size = 100000, min_seq_len=min_length, max_seq_len=max_length)

filename = "data/tsp_5_to_10_100000.pkl"
with open(filename, 'rb') as f:
    train_set = pickle.load(f)   # read file and build object

train_loader = DataLoader(dataset=train_set, batch_size=batch_size1, shuffle=True, collate_fn=sparse_seq_collate_fn)
print("加载训练数据集完成，开始加载测试数据集")

# test_set = TSPDataset(data_size = 10000, min_seq_len=min_length, max_seq_len=max_length)

filename = "data/tsp_5_to_10_10000.pkl"
with open(filename, 'rb') as f:
    test_set = pickle.load(f)   # read file and build object
test_loader = DataLoader(dataset=test_set, batch_size=batch_size1, shuffle=False, collate_fn=sparse_seq_collate_fn)
print("加载测试数据集完成")

正在加载训练数据集
加载训练数据集完成，开始加载测试数据集
加载测试数据集完成


In [8]:
emb_dim = 100
epochs = 100

model = PointerNet(input_dim=2, embedding_dim=emb_dim, hidden_size=emb_dim).to(device)

train_loss = AverageMeter()
train_accuracy = AverageMeter()
test_loss = AverageMeter()
test_accuracy = AverageMeter()

In [9]:
lr = 0.01
wd = 1e-5
optimizer = Adam(model.parameters(), lr=lr, weight_decay=wd)

def train():
    for epoch in range(epochs):
        # Train
        model.train()
        for batch_idx, (seq, length, target) in enumerate(train_loader):
            seq, length, target = seq.to(device), length.to(device), target.to(device)
            optimizer.zero_grad()
            log_pointer_score, argmax_pointer, mask = model(seq, length)

            # (batch * max_seq_len, max_seq_len)
            unrolled = log_pointer_score.view(-1, log_pointer_score.size(-1))
            # (batch_size, max_seq_len)
            loss = F.nll_loss(unrolled, target.view(-1), ignore_index=-1)
            assert not np.isnan(loss.item()), 'Model diverged with loss = NaN'

            loss.backward()
            optimizer.step()

            train_loss.update(loss.item(), seq.size(0))

            mask = mask[:, 0, :]
            train_accuracy.update(masked_accuracy(argmax_pointer, target, mask).item(), mask.int().sum().item())

            if batch_idx % 20 == 0:
                print('Epoch {}: Train [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.6f}'
                    .format(epoch, (batch_idx+1) * len(seq), len(train_loader.dataset),
                        100. * (batch_idx+1) / len(train_loader), train_loss.avg, train_accuracy.avg))

        # Test
        model.eval()
        for seq, length, target in test_loader:
            seq, length, target = seq.to(device), length.to(device), target.to(device)
            log_pointer_score, argmax_pointer, mask = model(seq, length)
            unrolled = log_pointer_score.view(-1, log_pointer_score.size(-1))
            loss = F.nll_loss(unrolled, target.view(-1), ignore_index=-1)
            assert not np.isnan(loss.item()), 'Model diverged with loss = NaN'

            test_loss.update(loss.item(), seq.size(0))

            mask = mask[:, 0, :]
            test_accuracy.update(masked_accuracy(argmax_pointer, target, mask).item(), mask.int().sum().item())
        print('Epoch {}: Test\tLoss: {:.6f}\tAccuracy: {:.6f}'.format(epoch, test_loss.avg, test_accuracy.avg))

In [10]:
train()

Epoch 0: Train [256/100000 (0%)]	Loss: 1.972573	Accuracy: 0.142143
Epoch 0: Train [5376/100000 (5%)]	Loss: 1.788150	Accuracy: 0.239590
Epoch 0: Train [10496/100000 (10%)]	Loss: 1.671765	Accuracy: 0.275194
Epoch 0: Train [15616/100000 (16%)]	Loss: 1.627388	Accuracy: 0.289528
Epoch 0: Train [20736/100000 (21%)]	Loss: 1.599244	Accuracy: 0.302996
Epoch 0: Train [25856/100000 (26%)]	Loss: 1.572684	Accuracy: 0.316795
Epoch 0: Train [30976/100000 (31%)]	Loss: 1.551924	Accuracy: 0.327271
Epoch 0: Train [36096/100000 (36%)]	Loss: 1.531676	Accuracy: 0.337647
Epoch 0: Train [41216/100000 (41%)]	Loss: 1.512225	Accuracy: 0.347609
Epoch 0: Train [46336/100000 (46%)]	Loss: 1.493422	Accuracy: 0.357576
Epoch 0: Train [51456/100000 (51%)]	Loss: 1.476308	Accuracy: 0.366317
Epoch 0: Train [56576/100000 (57%)]	Loss: 1.460354	Accuracy: 0.373958
Epoch 0: Train [61696/100000 (62%)]	Loss: 1.444453	Accuracy: 0.381467
Epoch 0: Train [66816/100000 (67%)]	Loss: 1.429063	Accuracy: 0.388503
Epoch 0: Train [71936/100

In [ ]:
filename = "model/model_5_to_10_100000_epoch_100_acc_65.3.pkl"
f = open(filename, 'wb')
pickle.dump(model, f)
f.close()